In [1]:
import joblib

In [2]:
gm = joblib.load('gmm_model.pkl')

C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator GaussianMixture from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
import cv2 
import mediapipe as mp 
import numpy as np 
import pandas as pd

In [5]:
#Build Keypoints using MP Holistic 
mp_holistic = mp.solutions.holistic # Holistic model 
mp_drawing = mp.solutions.drawing_utils # Drawing utilities 

In [6]:
def mediapipe_detection(image, model): 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB 
#     image.flags.writable = False                  # Image is no longer writable 
    results = model.process(image)                 # Make prediction 
#     image.flags.writable = True                   # Image is now writable  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR CONVERSION RGB 2 BGR 
    return image, results 
    
def draw_landmarks(image, results): 
    mp_drawing.draw_landmarks( 
      image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections 
    mp_drawing.draw_landmarks( 
      image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections 
      
def draw_styled_landmarks(image, results): 
    # Draw left hand connections 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,  
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),  
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2) 
                             )  
    # Draw right hand connections   
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,  
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),  
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                             )  

In [7]:
mapper = {}
mapper[0] = "A"
mapper[1] = "H"
mapper[2] = "Y"
mapper[3] = "S"
mapper[4] = "U"

In [8]:
#Main function 
cap = cv2.VideoCapture(0) 
# Set mediapipe model  
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
    while cap.isOpened(): 
  
        # Read feed 
        ret, frame = cap.read() 
        
        frame = cv2.flip(frame,1)
        
        # Make detections 
        image, results = mediapipe_detection(frame, holistic) 
        print(results) 
          
        # Draw landmarks 
        draw_styled_landmarks(image, results) 
  
        # Show to screen 
        cv2.imshow('OpenCV Feed', image) 
          
        zero_subs = [np.array([0, 0]) for _ in range(21)]
        
        # As I have mirrored the camera frame , left_hand_landmarks contains information for right hand
        # I will only use right hand ASL sign language 
        rh = (np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark])) if results.left_hand_landmarks else zero_subs
        
        rh = [(rh[i]-rh[0]).tolist() for i in range(1,len(rh))]
        if(rh!=zero_subs):
            rh.insert(0, [0, 0])
        
        rh = np.array(rh)
        rh = rh.flatten()
        rh = rh[2:]
        rh = rh.reshape(1, -1)
        print(rh)
        
        if not np.all(rh == 0):
            if (np.max(gm.predict_proba(rh))>0.99):
                print(mapper[np.argmax(gm.predict_proba(rh))])
                cv2.putText(img=image, text=mapper[np.argmax(gm.predict_proba(rh))], org=(150, 250), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=3, color=(0, 255, 0),thickness=3)
        cv2.imshow('OpenCV Feed', image) 
        # Break gracefully 
        if cv2.waitKey(50) & 0xFF == ord('q'): 
            break
    cap.release() 
    cv2.destroyAllWindows()

C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]

C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature nam

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[-3.27101350e-02 -9.57798362e-02 -6.04391098e-05 -2.11261630e-01
   5.59965968e-02 -2.87149668e-01  9.36517715e-02 -3.48616242e-01
   4.64426279e-02 -2.79037297e-01  9.63635445e-02 -3.81098211e-01
   1.35997295e-01 -4.38107550e-01  1.75463915e-01 -4.85076994e-01
   8.47628117e-02 -2.41744995e-01  1.35090768e-01 -3.50829482e-01
   1.74943328e-01 -4.18949604e-01  2.16151595e-01 -4.69707757e-01
   1.13286555e-01 -1.87722266e-01  1.35157943e-01 -2.47603714e-01
   8.35253596e-02 -1.77715957e-01  5.14690876e-02 -1.21956348e-01
   1.36939108e-01 -1.26808584e-01  1.40267968e-01 -1.79799438e-01
   9.58855152e-02 -1.32287264e-01  6.67188764e-02 -9.41168070e-02]]
H
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[-0.0345459  -0.09309095 -0.00254631 -0.20832741  0.05489445 -0.28245723
   0.09157312 -0.34438431  0.04553241 -0.27876472  0.09566116 -0.38027191
   0.13431263 -0.43804798  0.17199969 -0.4856326   0.0832426  -0.24087656


C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[-3.22862864e-02 -9.13019776e-02 -1.43051147e-04 -2.08861351e-01
   5.50655127e-02 -2.86883593e-01  9.23771262e-02 -3.50856245e-01
   4.50479984e-02 -2.82684922e-01  9.59115028e-02 -3.83069992e-01
   1.34832919e-01 -4.38006520e-01  1.72904015e-01 -4.83932972e-01
   8.49760175e-02 -2.45080531e-01  1.32529020e-01 -3.53449583e-01
   1.71466053e-01 -4.18635517e-01  2.10811317e-01 -4.69078809e-01
   1.15055621e-01 -1.91544771e-01  1.33369446e-01 -2.47971833e-01
   8.35634470e-02 -1.78560913e-01  5.08033037e-02 -1.24063373e-01
   1.39889002e-01 -1.30316854e-01  1.40505135e-01 -1.77202702e-01
   9.52343941e-02 -1.33163214e-01  6.28503561e-02 -9.82819200e-02]]
H
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[-0.03322834 -0.09263635 -0.00247443 -0.2086696   0.05342108 -0.28435481
   0.09197909 -0.34597397  0.0444541  -0.28447813  0.09623092 -0.38710588
   0.13601619 -0.44433936  0.17456853 -0.49130216  0.08330661 -0.24598646


C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[[-3.28422189e-02 -9.30399299e-02 -3.50475311e-05 -2.10822821e-01
   5.78057170e-02 -2.90223956e-01  9.98893976e-02 -3.50973487e-01
   4.75749373e-02 -2.82343626e-01  1.00924492e-01 -3.83792996e-01
   1.41481042e-01 -4.39915717e-01  1.80962086e-01 -4.87279743e-01
   8.66062045e-02 -2.43201256e-01  1.38737500e-01 -3.54321063e-01
   1.78688526e-01 -4.20824140e-01  2.19270706e-01 -4.70779777e-01
   1.15356266e-01 -1.87249362e-01  1.37783468e-01 -2.45546103e-01
   8.70159268e-02 -1.77679181e-01  5.42302132e-02 -1.23547256e-01
   1.38752818e-01 -1.24205172e-01  1.41974688e-01 -1.74633384e-01
   9.75759029e-02 -1.28650665e-01  6.69536591e-02 -9.15282965e-02]]
H
